In [25]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import csv
from scipy.optimize import minimize 
from scipy import integrate
from datetime import datetime


In [2]:
#population measured by 1,000
#population_density measured by person/square kilo
#electricity_consumption by Gigawatthour/year
#GDP measured by billion usd
dataframe=pd.read_excel("mcm_city_data.xlsx")
dataset=dataframe.values.tolist()
num=len(dataset)

def L(parameter,population,population_density,electricity_consumption,GDP):
    a=parameter[0]
    b=parameter[1]
    c=parameter[2]
    k=parameter[3]
    bracket_factor=[1,1.1,1.3,1.5,1.8,2.5]
    bracket_popuation=[5,20,50,100,500,1000]
    bracket=0
    for i in range(len(bracket_factor)-1):
        if population>bracket_popuation[i]:
            bracket+=1
        else:
            break
    return max(0.1,k*bracket_factor[bracket]*(electricity_consumption**(a)*GDP**(b)*population**(c/(1+200/population_density))))

def error(parameter):
    output=0
    for i in range(num):
        predict=L(parameter,dataset[i][5],dataset[i][6],dataset[i][4],dataset[i][8])
        actual=dataset[i][9]
        output+=((predict-actual)/(min(predict,actual))+0.01)**2
    return output
b=(0,20)
mybounds=(b,b,b,b)
a=minimize(error,[0.2,0.1,0.05,12],bounds=mybounds)
# print(a)

#therefore, we choose a=0.16,b=0.08,c=0.02,k=3.42
def L_final(population,population_density,electricity_consumption,GDP):
    a=0.16
    b=0.08
    c=0.02
    k=3.42
    bracket_factor=[1,1.1,1.3,1.5,1.8,2.5]
    bracket_popuation=[5,20,50,100,500,1000]
    bracket=0
    for i in range(len(bracket_factor)-1):
        if population>bracket_popuation[i]:
            bracket+=1
        else:
            break
    return max(0.1,k*bracket_factor[bracket]*(electricity_consumption**(a)*GDP**(b)*population**(c/(1+200/population_density))))


In [108]:
my_map={}
for i in range(num):
    my_map[dataset[i][0]]=L_final(dataset[i][5],dataset[i][6],dataset[i][4],dataset[i][8])
# with open ("output.csv","w") as out:
#     writer=csv.writer(out,delimiter=",")
#     writer.writerow(["predicted","Actual"])
#     for i in range(num):
#         a=L_final(dataset[i][5],dataset[i][6],dataset[i][4],dataset[i][8])
#         b=dataset[i][9]
#         writer.writerow([a,b])
#     out.close()
my_map["test"]=70

In [4]:
group_proportion=[0.12,0.19,0.19,0.22,0.17,0.1]
group_wavelength=[425,475,525,575,625,675]



In [13]:
#constants: c1,c2,P,Q,Ka,KO,KN
c1=24/(np.pi)**3
c2=1.4
P=0.4
Q_a=0.3
Q_s=1.7
Q_w=0.6
K_a=1
K_o=1
K_n=1
def L_point(L0,r,R):
    if r<R/2:
        return 16/5*L0
    else:
        return 16/5*L0*np.exp(-9.53/R*(r-R/2))
def L_up(L0,R,r0,psi,P,Q):
    return r0*L_point(L0,r0,R)*(24/((np.pi)**3)*P*psi**(2)+1.4*(1-P)*Q*np.cos(psi))
def Li_reflect(L0,R,d,i,altitude,Q,P,r,theta,psi,r0,theta0):
    OT=np.array([d-r0*np.cos(theta0),-r0*np.sin(theta0),0])
    OA=np.array([r*np.sin(psi)*np.cos(theta),r*np.sin(psi)*np.sin(theta),r*np.cos(psi)])
    TA=OA-OT
    cos_alpha=np.dot(OA,TA)/(np.linalg.norm(OA)*np.linalg.norm(TA))
    d1_sq=r**2
    d2_sq=np.dot(TA,TA)
    h=r*np.cos(psi)
    N_a=(0.6*10**2)*np.exp((-altitude-h)/(1000))
    N_n=2.04*10**(7-1.13*10**(-4)*(h+altitude))
    N_o=0.51*10**(7-1.3*10**(-4)*(h+altitude))
    sigma_a=K_a*(cos_alpha+0.5*(3*cos_alpha**(2)+1))/(d1_sq+d2_sq)
    sigma_o=K_o*8*np.pi**4*(cos_alpha**2+1)/((d1_sq+d2_sq)*group_wavelength[i]**4)
    sigma_n=sigma_o/K_o*K_n
    out1=L_up(L0,R,r0,psi,P,Q)
    A=sigma_a*N_a
    O=sigma_o*N_o
    N=sigma_n*N_n
    return out1*(A+O+N)
def city_impact_i(i,name,is_coastal,month_snow,altitude,area,distance):
    L0=group_proportion[i]*my_map[name]
    R=math.sqrt(area/np.pi)*1000
    Q= Q_w if is_coastal else Q_a
    Q+=month_snow/12*Q_s
    P=0.4
    #use random method
    sample_size=10
    core_impact=[]
    sur_impact=[]
    
    #bounds for integration and options
    def limit_theta(psi,r):
        return [0,np.pi*2]
    def limit_psi(r):    
        return [0.1,np.pi/2-0.1]
    def limit_r():
        return [2000,10000]
    options={"epsabs":10,"limit":3}
    
    for p in range(sample_size):
        r0=np.random.uniform(0,R)
        theta0=np.random.uniform(0,2*np.pi)
        def target(theta,psi,r):
            return Li_reflect(L0,R,distance*1000,i,altitude,Q,P,r,theta,psi,r0,theta0)
        if (r0<R/2):
            core_impact.append(integrate.nquad(target,[limit_theta,limit_psi,limit_r],opts=options)[0])
        else:
            sur_impact.append(integrate.nquad(target,[limit_theta,limit_psi,limit_r],opts=options)[0])
    return (sum(core_impact)+sum(sur_impact))/sample_size
        
#     def target(r,theta,psi,r0,theta0):
#         return L_reflect(L0,R,distance,i,altitude,Q,P,r,theta,psi,r0,theta0)
#     def limit_r(theta,psi,r0,theta0):
#         return [1000,20000]
#     def limit_theta(psi,r0,theta0):
#         return [0,np.pi*2]
#     def limit_psi(r0,theta0):
#         return [0,np.pi/2]
#     def limit_r0(theta0):
#         return [0,R]
#     def limit_theta0():
#         return [0,np.pi*2]
#     return integrate.nquad(target,[limit_r,limit_theta,limit_psi,limit_r0,limit_theta0])

In [14]:
mark=datetime.now()
print(city_impact_i(0,"New York",1,2,0,469,22))
print(city_impact_i(0,"New York",1,2,0,469,30))
print(city_impact_i(0,"New York",1,2,0,469,75))
print(datetime.now()-mark)

89.54931333188298
44.20370583827555
6.23129273434656
0:03:41.362820


In [21]:
def yo(g):
    a=1
    def hi(y,x):
        return np.cos(y**2+x**2)
    def boundy(x):
        return [-math.sqrt(1-x**2),0]
    return integrate.nquad(hi,[boundy,[-a,a]])
yo(1)

(1.3217795320408217, 3.85837806149425e-09)

In [111]:
def L_point(L0,r,R):
    if r<R/2:
        return 16/5*L0
    else:
        return 16/5*L0*np.exp(-9.53/R*(r-R/2))
def L_up(L0,R,r0,psi,P,Q):
    return r0*L_point(L0,r0,R)*(24/((np.pi)**3)*P*psi**(2)+1.4*(1-P)*Q*np.cos(psi))
def Li_reflect_t(K_a,K_o,K_n,L0,R,d,i,altitude,Q,P,r,theta,psi,r0,theta0):
    OT=np.array([d-r0*np.cos(theta0),-r0*np.sin(theta0),0])
    OA=np.array([r*np.sin(psi)*np.cos(theta),r*np.sin(psi)*np.sin(theta),r*np.cos(psi)])
    TA=OA-OT
    cos_alpha=np.dot(OA,TA)/(np.linalg.norm(OA)*np.linalg.norm(TA))
    d1_sq=r**2
    d2_sq=np.dot(TA,TA)
    h=r*np.cos(psi)
    N_a=(0.6*10**2)*np.exp((-altitude-h)/(1000))
    N_n=2.04*10**(7-1.13*10**(-4)*(h+altitude))
    N_o=0.51*10**(7-1.3*10**(-4)*(h+altitude))
    sigma_a=K_a*(cos_alpha+0.5*(3*cos_alpha**(2)+1))/(d1_sq+d2_sq)
    sigma_o=K_o*8*np.pi**4*(cos_alpha**2+1)/((d1_sq+d2_sq)*group_wavelength[i]**4)
    sigma_n=sigma_o/K_o*K_n
    out1=L_up(L0,R,r0,psi,P,Q)
    A=sigma_a*N_a
    O=sigma_o*N_o
    N=sigma_n*N_n
    return out1*(A+O+N)
def city_impact_t(K_a,K_o,K_n,i,name,is_coastal,month_snow,altitude,area,distance):
    L0=group_proportion[i]*my_map[name]
    R=math.sqrt(area/np.pi)*1000
    Q= Q_w if is_coastal else Q_a
    Q+=month_snow/12*Q_s
    P=0.4
    #use random method
    sample_size=10
    #bounds for integration and options
    def limit_theta(psi,r):
        return [0,np.pi*2]
    def limit_psi(r):    
        return [0,np.pi/2-0.1]
    def limit_r():
        return [2000,10000]
    options={"epsabs":20,"limit":3}
    data=0
    for p in range(sample_size):
        r0=np.random.uniform(0,R)
        theta0=np.random.uniform(0,2*np.pi)
        def target_t(theta,psi,r):
            return Li_reflect_t(K_a,K_o,K_n,L0,R,distance*1000,i,altitude,Q,P,r,theta,psi,r0,theta0)
        data+=integrate.nquad(target_t,[limit_theta,limit_psi,limit_r],opts=options)[0]
    return data/sample_size

num=25
hi=list(range(num))
cao=[]
for current in range(6):
    work=np.zeros(num)
    for j in range(num):
        work[j]=city_impact_t(0.2,0.2,0.2,current,"test",1,0,0,52,hi[j]+5)
    cao.append(work)
        

In [93]:
dataframe2=pd.read_excel("mcm_impact_data.xlsx")
trainingdata=dataframe2.values.tolist()
sampnum=len(trainingdata)
def loss_function(K):
    K_a=K[0]
    K_o=K[1]
    K_n=K[2]
    loss=0
    with open ("training.csv", "w") as  output:
        writer=csv.writer(output,delimiter=",")
        writer.writerow(["Guess","Anwer"])
        for n in range(sampnum):
            print("currently working on"+trainingdata[n][0])
            guess=0
            for j in range(len(group_proportion)):
                guess+=city_impact_t(K_a,K_o,K_n,j,trainingdata[n][0],trainingdata[n][1],trainingdata[n][2],trainingdata[n][3],trainingdata[n][4],trainingdata[n][5])
    #         print("Our prediction:"+str(guess))
            answer=trainingdata[n][6]
    #         print("Answer:"+str(answer))
            loss+=(guess-answer)**(2)/(min(guess,answer))
            writer.writerow([guess,answer])
        output.close()
    print("K_a="+str(K_a))
    print("K_o="+str(K_o))
    print("K_n="+str(K_n))
    print(loss)
    return loss
boundary=(0,2)
my_boundaries=[boundary,boundary,boundary]
my_requirements={"maxiter":1}
final=minimize(loss_function,[0.2,0.2,0.2],method='BFGS',bounds=my_boundaries,tol=100,options=my_requirements)


currently working onNew York
currently working onNew York
currently working onNew York
currently working onLos Angeles
currently working onLos Angeles
currently working onLos Angeles
currently working onChicago
currently working onChicago
currently working onChicago
currently working onHouston
currently working onHouston
currently working onHouston
currently working onPhenix
currently working onPhenix
currently working onPhenix
currently working onCharlotte
currently working onCharlotte
currently working onCharlotte
currently working onColumbus
currently working onColumbus
currently working onColumbus
currently working onAlbuqueue
currently working onAlbuqueue
currently working onAlbuqueue
currently working onMadison
currently working onMadison
currently working onMadison
currently working onBozeman
currently working onBozeman
currently working onBozeman
currently working onNorth Platte
currently working onNorth Platte
currently working onNorth Platte
currently working onCarbondale
cur

currently working onLa Paz
currently working onSan Paulo
currently working onSan Paulo
currently working onValparíaso
currently working onValparíaso
currently working onPuyo
currently working onLagos
currently working onLagos
currently working onKigali
currently working onGaborone
currently working onAbidjan
currently working onAbidjan
currently working onTokyo
currently working onTokyo
currently working onNagasaki
currently working onHualien City
currently working onHualien City
currently working onBattambang
currently working onHpa-An
currently working onParis
currently working onParis
currently working onParis
currently working onReykjaik
currently working onReykjaik
currently working onMalmo
currently working onMalmo
currently working onSiena
K_a=0.2
K_o=0.2
K_n=0.2000000149011612
114437.43981003872
currently working onNew York
currently working onNew York
currently working onNew York
currently working onLos Angeles
currently working onLos Angeles
currently working onLos Angeles
cu

currently working onColumbus
currently working onColumbus
currently working onColumbus
currently working onAlbuqueue
currently working onAlbuqueue
currently working onAlbuqueue
currently working onMadison
currently working onMadison
currently working onMadison
currently working onBozeman
currently working onBozeman
currently working onBozeman
currently working onNorth Platte
currently working onNorth Platte
currently working onNorth Platte
currently working onCarbondale
currently working onCarbondale
currently working onTraverse City
currently working onTraverse City
currently working onVancouver 
currently working onVancouver 
currently working onVancouver 
currently working onHalifax
currently working onHalifax
currently working onEdmonton
currently working onEdmonton
currently working onRegina
currently working onRegina
currently working onSherbrooke
currently working onSherbrooke
currently working onLa Paz
currently working onLa Paz
currently working onSan Paulo
currently working o

currently working onHualien City
currently working onHualien City
currently working onBattambang
currently working onHpa-An
currently working onParis
currently working onParis
currently working onParis
currently working onReykjaik
currently working onReykjaik
currently working onMalmo
currently working onMalmo
currently working onSiena
K_a=0.3088717456492325
K_o=0.4682291995881939
K_n=0.21473113002838304
145024.9638429103
currently working onNew York
currently working onNew York
currently working onNew York
currently working onLos Angeles
currently working onLos Angeles
currently working onLos Angeles
currently working onChicago
currently working onChicago
currently working onChicago
currently working onHouston
currently working onHouston
currently working onHouston
currently working onPhenix
currently working onPhenix
currently working onPhenix
currently working onCharlotte
currently working onCharlotte
currently working onCharlotte
currently working onColumbus
currently working onCol

currently working onNorth Platte
currently working onNorth Platte
currently working onCarbondale
currently working onCarbondale
currently working onTraverse City
currently working onTraverse City
currently working onVancouver 
currently working onVancouver 
currently working onVancouver 
currently working onHalifax
currently working onHalifax
currently working onEdmonton
currently working onEdmonton
currently working onRegina
currently working onRegina
currently working onSherbrooke
currently working onSherbrooke
currently working onLa Paz
currently working onLa Paz
currently working onSan Paulo
currently working onSan Paulo
currently working onValparíaso
currently working onValparíaso
currently working onPuyo
currently working onLagos
currently working onLagos
currently working onKigali
currently working onGaborone
currently working onAbidjan
currently working onAbidjan
currently working onTokyo
currently working onTokyo
currently working onNagasaki
currently working onHualien City
cu

currently working onNew York
currently working onNew York
currently working onLos Angeles
currently working onLos Angeles
currently working onLos Angeles
currently working onChicago
currently working onChicago
currently working onChicago
currently working onHouston
currently working onHouston
currently working onHouston
currently working onPhenix
currently working onPhenix
currently working onPhenix
currently working onCharlotte
currently working onCharlotte
currently working onCharlotte
currently working onColumbus
currently working onColumbus
currently working onColumbus
currently working onAlbuqueue
currently working onAlbuqueue
currently working onAlbuqueue
currently working onMadison
currently working onMadison
currently working onMadison
currently working onBozeman
currently working onBozeman
currently working onBozeman
currently working onNorth Platte
currently working onNorth Platte
currently working onNorth Platte
currently working onCarbondale
currently working onCarbondale
c

currently working onSherbrooke
currently working onSherbrooke
currently working onLa Paz
currently working onLa Paz
currently working onSan Paulo
currently working onSan Paulo
currently working onValparíaso
currently working onValparíaso
currently working onPuyo
currently working onLagos
currently working onLagos
currently working onKigali
currently working onGaborone
currently working onAbidjan
currently working onAbidjan
currently working onTokyo
currently working onTokyo
currently working onNagasaki
currently working onHualien City
currently working onHualien City
currently working onBattambang
currently working onHpa-An
currently working onParis
currently working onParis
currently working onParis
currently working onReykjaik
currently working onReykjaik
currently working onMalmo
currently working onMalmo
currently working onSiena
K_a=0.2013884792439467
K_o=0.20342078338517944
K_n=0.20018786920932863
102986.84029845642
currently working onNew York
currently working onNew York
curren

currently working onPhenix
currently working onCharlotte
currently working onCharlotte
currently working onCharlotte
currently working onColumbus
currently working onColumbus
currently working onColumbus
currently working onAlbuqueue
currently working onAlbuqueue
currently working onAlbuqueue
currently working onMadison
currently working onMadison
currently working onMadison
currently working onBozeman
currently working onBozeman
currently working onBozeman
currently working onNorth Platte
currently working onNorth Platte
currently working onNorth Platte
currently working onCarbondale
currently working onCarbondale
currently working onTraverse City
currently working onTraverse City
currently working onVancouver 
currently working onVancouver 
currently working onVancouver 
currently working onHalifax
currently working onHalifax
currently working onEdmonton
currently working onEdmonton
currently working onRegina
currently working onRegina
currently working onSherbrooke
currently working

KeyboardInterrupt: 

In [82]:
# def min_method(fn, grad, x0):
#     all_fn = [[fn(x0).item(),0]]
#     def store(x, gfk,gnorm):  # callback function
#         print(([fn(x).item(),gfk,gnorm]))

#     ans = minimize(fn, x0, method='CG', jac=grad, callback=store,
#                    options={'disp': True, 'gtol': 1e-06,"maxiter":1})
#     return ans, all_fn

# def hi(A):
#     return A[0]*A[1]/A[2]
# x0 = np.array([0.2,0.2,0.2], dtype=np.double)
# print(min_method(hi, '3-point',x0))

TypeError: store() missing 2 required positional arguments: 'gfk' and 'gnorm'

In [113]:
print(cao)
dataset_1=[38.80173372, 21.3520707 , 15.44932793, 14.71991844,  8.44830968,
       11.46720083, 10.09565123,  4.81855944,  7.05816997,  6.61899111,
        4.90147752,  6.28239719,  4.40385212,  3.48684537,  2.07967261,
        2.93718716,  2.25397225,  2.15257213,  2.2294911 ,  1.96584868,
        1.7521848 ,  1.9029647 ,  1.46268465,  2.40321557,  1.84294157]
dataset_2=[49.8283657 , 40.9589843 , 24.18808506, 38.22513319, 22.99934924,
       17.76311396, 15.9884597 ,  9.52707732,  6.96150015,  9.52586099,
        6.811817  ,  5.29164667,  6.5616322 ,  3.69238902,  3.9939223 ,
        4.58773831,  3.88783721,  2.704488  ,  4.1265414 ,  3.35807632,
        2.36449789,  3.12596054,  2.12273918,  1.85765429,  2.53950551]
dataset_3=[55.75109642, 36.2488358 , 52.73938356, 34.58515447, 16.66301399,
       14.86276607, 10.00223917, 11.24049829, 10.81612664, 10.52360666,
        9.06418972,  7.34667483,  6.68002212,  4.11438076,  5.76921942,
        3.98085087,  4.6400965 ,  3.91228732,  4.34194473,  3.50698055,
        3.65899261,  2.11975964,  2.22115903,  2.94350836,  1.4683771]
dataset_4=[63.50806599, 37.88628859, 33.14084372, 26.24747288, 31.31288474,
       13.8417116 , 16.2589916 , 13.6733722 ,  7.0695459 ,  9.86063558,
       10.94389933,  5.62925872,  4.94156058,  4.05799762,  6.7444371 ,
        5.37587355,  3.67333961,  5.06442324,  3.79037728,  3.18082423,
        3.6372988 ,  3.23168502,  2.96021057,  1.56382171,  2.76096973]
daatset_5=[53.90530259, 33.10606563, 23.92099304, 32.31562024,  7.88543081,
       12.9894271 , 11.96993555,  7.67006252,  8.58826962,  7.81277439,
        8.56928705,  4.96686837,  5.98515083,  5.81335587,  4.38876251,
        3.22665135,  3.40284359,  4.03044746,  3.30855727,  3.11411774,
        1.35680056,  3.11962332,  1.89859639,  2.51861731,  2.13795185]
dataset_6=[15.95572559, 19.08800359, 13.90438904, 11.3406259 ,  9.53559967,
        7.70374332,  8.36756678,  4.96621018,  5.4941164 ,  4.92077398,
        4.7516974 ,  3.77300533,  3.13796761,  2.80141348,  3.34625559,
        1.37023662,  2.82325919,  2.10890031,  1.46202745,  1.22316116,
        2.12822277,  1.27397349,  1.38665912,  0.86494641,  1.19178792]

[array([38.80173372, 21.3520707 , 15.44932793, 14.71991844,  8.44830968,
       11.46720083, 10.09565123,  4.81855944,  7.05816997,  6.61899111,
        4.90147752,  6.28239719,  4.40385212,  3.48684537,  2.07967261,
        2.93718716,  2.25397225,  2.15257213,  2.2294911 ,  1.96584868,
        1.7521848 ,  1.9029647 ,  1.46268465,  2.40321557,  1.84294157]), array([49.8283657 , 40.9589843 , 24.18808506, 38.22513319, 22.99934924,
       17.76311396, 15.9884597 ,  9.52707732,  6.96150015,  9.52586099,
        6.811817  ,  5.29164667,  6.5616322 ,  3.69238902,  3.9939223 ,
        4.58773831,  3.88783721,  2.704488  ,  4.1265414 ,  3.35807632,
        2.36449789,  3.12596054,  2.12273918,  1.85765429,  2.53950551]), array([55.75109642, 36.2488358 , 52.73938356, 34.58515447, 16.66301399,
       14.86276607, 10.00223917, 11.24049829, 10.81612664, 10.52360666,
        9.06418972,  7.34667483,  6.68002212,  4.11438076,  5.76921942,
        3.98085087,  4.6400965 ,  3.91228732,  4.34194473, 